In [1]:
import os

# # Set the environment variable to make only GPU 0 visible
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(gpus)

2025-02-26 09:47:51.208755: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.backend as K



In [3]:
# Set the environment variable to make only GPU 0 visible
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Now, when TensorFlow runs, it should only see GPU 0
gpus = tf.config.list_physical_devices('GPU')
print(gpus)


if gpus:
    try:
        # Set memory growth to avoid allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        # Optionally, limit GPU memory usage to a fraction (e.g., 33%)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=40960)])  # Set memory limit in MB
    except RuntimeError as e:
        print(e)

# Now, you can create a session (if using TF 1.x compatibility mode)
if tf.__version__.startswith('1.'):
    sess = tf.Session()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
def get_data_phase1(LABEL_FIST_HALF):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train = x_train/255.0
    x_test = x_test/255.0

    #train_val_ratio = 0.8
    #train_num = int(x_train_val.shape[0]*train_val_ratio)
    
    train_filter = np.where(np.in1d(y_train, LABEL_FIST_HALF))
    test_filter =  np.where(np.in1d(y_test, LABEL_FIST_HALF))
    
    x_train = x_train[train_filter]
    y_train = y_train[train_filter]
    x_test = x_test[test_filter]
    y_test = y_test[test_filter]

    Y_train = tf.keras.utils.to_categorical(y_train, 10)
    Y_test = tf.keras.utils.to_categorical(y_test, 10)
    
    return x_train,x_test,Y_train,Y_test

def getnoisevariance(SNR,rate,P=1):
    # the SNR in args[0] is actually EbN0
    snrdB = SNR + 10*np.log10(rate)
    snr = 10.0**(snrdB/10.0)
    #P_avg = 1
    N0 = P/snr
    return (N0/2)

In [5]:
label_permutation = np.arange(10)
LABEL_FIRST_HALF = label_permutation[:5]

x_train,x_test,Y_train,Y_test = get_data_phase1(LABEL_FIRST_HALF)

In [6]:
label2_list

NameError: name 'label2_list' is not defined

In [7]:
label2_list = np.zeros((6,5))
for i in range(6):
    label2_list[i][:] = label_permutation[i:i+5]

In [9]:
label2_list[5-0]

array([5., 6., 7., 8., 9.])

# model

In [14]:
### normalized Input before 
class GaussianNoiseLayer(layers.Layer):
    def __init__(self, stddev, **kwargs):
        super(GaussianNoiseLayer, self).__init__(**kwargs)
        self.stddev = stddev
    
    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
            return inputs + noise
        else:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
            return inputs + noise
    
    def get_config(self):
        config = super(GaussianNoiseLayer, self).get_config()
        config.update({'noise_var': self.stddev})
        return config

In [8]:
### setting 
SAMPLE_NUM = 15000
lambda_list = [0,1,2,5,10]
SNR_train_list = [0,5,10,15]
SNR_test_list = np.arange(-5,20,3)
acc_list = []
datalist = []

In [9]:
from tensorflow.keras.models import load_model

embeddingDim = 80

batchsize = 256

for snr_train in [15]:
    print("current training snr: ",snr_train)
    for lambda_val in lambda_list:
        noise_sd = getnoisevariance(snr_train,1)
        print("current lambda: ",lambda_val )
        K.clear_session()
        tf.keras.utils.set_random_seed(42)

        input_layer = layers.Input(shape=(32, 32, 3))

        # Increase the depth and number of filters
        encoder_1 = layers.Conv2D(96, (3, 3), activation='relu', padding='same')(input_layer)
        encoder_1 = layers.BatchNormalization()(encoder_1)
        encoder_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_1)
        encoder_2 = layers.BatchNormalization()(encoder_2)
        maxpool_2 = layers.MaxPooling2D((2, 2))(encoder_2)

        # Reduce dimensionality with another convolution
        encoder_3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(maxpool_2)
        encoder_3 = layers.BatchNormalization()(encoder_3)
        maxpool_3 = layers.MaxPooling2D((2, 2))(encoder_3)

        # Normalization and noise
        normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(maxpool_3)
        noise_layer = GaussianNoiseLayer(stddev=0.1)(normalized_x)

        # CE branch
        CE_cnn_1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
        CE_cnn_1 = layers.BatchNormalization()(CE_cnn_1)
        CE_cnn_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(CE_cnn_1)
        CE_cnn_2 = layers.BatchNormalization()(CE_cnn_2)
        flatten_1 = layers.GlobalAveragePooling2D()(CE_cnn_2)
        CE_dense_2 = layers.Dense(256, activation='relu')(flatten_1)
        CE_dense_2 = layers.Dropout(0.5)(CE_dense_2)

        CE_output = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_dense_2)

        # MSE branch
        mse_1 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(noise_layer)
        upsample1 = layers.UpSampling2D((2, 2))(mse_1)
        mse_2 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(upsample1)
        upsample2 = layers.UpSampling2D((2, 2))(mse_2)  # Additional upsampling layer to match the original input size
        mse_output = layers.Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same', name='mse')(upsample2)

        model = tf.keras.Model(inputs=input_layer, outputs=[CE_output, mse_output])

        # Adjust learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-2,  # Slightly lower learning rate
            decay_steps=10000,
            decay_rate=0.8)
        opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

        model.compile(optimizer=opt,
                      loss={'CE': 'categorical_crossentropy', 'mse': 'mse'},
                      metrics={'CE': 'accuracy', 'mse': tf.keras.metrics.RootMeanSquaredError()},
                      loss_weights=[1, lambda_val])

        history = model.fit(x=x_train, y=(Y_train, x_train),
                            batch_size=256,
                            epochs=200,
                            verbose=0,
                            validation_data=(x_test, (Y_test, x_test)))

        first_acc = history.history['val_CE_accuracy'][-1]
        print("current acc: ",first_acc)
        print("training phase 2")

        #model.save("middle.h5")

        for i in range(label2_list.shape[0]):
            print("current overlap: ",(i))
            x_train_2,x_test_2,Y_train_2,Y_test_2 = get_data_phase1(label2_list[5-i])
            for snr_test in SNR_test_list: 
                print("test snr: ", snr_test)
                noise_sd_test = getnoisevariance(snr_test,1)
    #             embedding_network_2 = load_model("embedding_network.h5")
    #             for layer in embedding_network_2.layers:
    #                 layer.trainable = False

                CE_cnn_3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
                CE_cnn_3 = layers.BatchNormalization()(CE_cnn_3)
                CE_cnn_4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(CE_cnn_3)
                CE_cnn_4 = layers.BatchNormalization()(CE_cnn_4)
                flatten_2 = layers.GlobalAveragePooling2D()(CE_cnn_4)
                CE_dense_3 = layers.Dense(256, activation='relu')(flatten_2)
                CE_dense_3 = layers.Dropout(0.5)(CE_dense_3)
                CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE2')(CE_dense_3)
    
                reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])
                reconstructed_model.layers[1].trainable = False
                reconstructed_model.layers[2].trainable = False
                #reconstructed_model.layers[3].trainable = False


                lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=3e-2,
                decay_steps=10000,
                decay_rate=0.8)
                opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
                # Compile the models

                reconstructed_model.compile(optimizer=opt, 
                            loss = {'CE2' : 'categorical_crossentropy', 
                                      'mse' : 'mse'
                                     },              loss_weights=[1, 0],
                              metrics = {'CE2' : 'accuracy', 
                                         'mse': "mean_squared_error"
                                       }
                             )

                history1 = reconstructed_model.fit(x=x_train_2, y=(Y_train_2,x_train_2),
                                    batch_size=256,
                                    epochs=100,
                                    verbose=0,
                                    validation_data=(x_test_2, (Y_test_2,x_test_2)))

                results = reconstructed_model.evaluate(x = x_test_2, y = (Y_test_2,x_test_2))
                acc_list.append(results[3])

                savedata = {"lambda":lambda_val,
                            "first lr":0.01,
                            "first batchsize":256,
                            "first accuracy":first_acc,
                            "train snr":snr_train,
                            "test snr":snr_test,
                            "second loss":results[4],
                            "second accuracy":results[3]}

                datalist.append(savedata)

                print("final acc:", results[3])



current training snr:  0
current lambda:  0


2025-02-24 08:40:51.595330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1138 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2025-02-24 08:40:55.334071: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2025-02-24 08:40:55.551262: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 442.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-02-24 08:40:55.555814: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 433.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-02-24 0

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d_1/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    F

In [ ]:
label2_list

In [33]:

df = pd.DataFrame(datalist)

# Define the file path and name
file_path = 'RLA_cifar_new_snr0.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(file_path, index=False)

In [ ]:
from tensorflow.keras.models import load_model

embeddingDim = 80

batchsize = 256

for snr_train in SNR_train_list:
    print("current training snr: ",snr_train)
    for lambda_val in lambda_list:
        noise_sd = getnoisevariance(snr_train,1)
        print("current lambda: ",lambda_val )
        K.clear_session()
        tf.keras.utils.set_random_seed(42)

        input_layer  = layers.Input(shape=(32, 32, 3))
        encoder_1 = layers.Conv2D(96, (3, 3), activation='relu', padding='same')(input_layer)
        #maxpool_1 = layers.MaxPooling2D((2, 2))(encoder_1)
        encoder_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_1)
        maxpool_2 = layers.MaxPooling2D((2, 2))(encoder_2)
        #x = layers.Dropout(0.25)(maxpool_2)
        encoder_2_quantized = tf.keras.layers.Lambda(
            lambda x: tf.quantization.fake_quant_with_min_max_vars(x, min=0.0, max=6.0, num_bits=8)
        )(maxpool_2)
        
        embedding_network = tf.keras.Model(inputs=input_layer, outputs=encoder_2_quantized)

        normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2_quantized)
        noise_layer = GaussianNoiseLayer(stddev = noise_sd)(normalized_x)

        CE_cnn_1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
        flatten_1 = layers.Flatten()(CE_cnn_1)
        #bn = layers.BatchNormalization()(flatten_1)
        CE_dense_2 = layers.Dense(256, activation='relu')(flatten_1)
        #CE_dense_2 = layers.Dense(256, activation='relu')(CE_dense_2)
        #CE_dense_3 = layers.Dense(128, activation='relu')(CE_dense_2)
        #CE_dense_4 = layers.Dense(100, activation='relu')(CE_dense_3)

        CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_2)
        #CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

        #dense_2 = layers.Dense(64, activation='relu')(flatten_1)
        mse_1 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(noise_layer)
        upsample1 = layers.UpSampling2D((2,2))(mse_1)
        mse_2 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(upsample1)
        #upsample2 = layers.UpSampling2D((2,2))(mse_2)
        mse_output = layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same',name='mse')(mse_2)

        model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-2,
        decay_steps=10000,
        decay_rate=0.8)
        opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        model.compile(optimizer=opt,
                      loss = {'CE' : 'categorical_crossentropy', 
                              'mse' : 'mse'},
                      metrics = {'CE' : 'accuracy', 
                                 'mse': tf.keras.metrics.RootMeanSquaredError()},
                      loss_weights=[1, lambda_val])

        history = model.fit(x=x_train, y=(Y_train,x_train),
                            batch_size=256,
                            epochs=200,
                            verbose=0,
                            validation_data=(x_test, (Y_test,x_test)))
        
        first_acc = history.history['val_CE_accuracy'][-1]
        print("current acc: ",first_acc)
        print("training phase 2")
        
        embedding_network.save("embedding_network2.h5")


        #model.save("middle.h5")

        for i in range(label2_list.shape[0]):
            print("current overlap: ",(i))
            x_train_2,x_test_2,Y_train_2,Y_test_2 = get_data_phase1(label2_list[5-i])
            for snr_test in SNR_test_list: 
                print("test snr: ", snr_test)
                noise_sd_test = getnoisevariance(snr_test,1)
                
                #embedding_network_2 = load_model("embedding_network2.h5")
                embedding_network_2 = tf.keras.models.load_model("embedding_network2.h5", compile=False)
                for layer in embedding_network_2.layers:
                    layer.trainable = False
                    
                noise_layer_1 = GaussianNoiseLayer(stddev = noise_sd_test)(normalized_x)
                CE_cnn_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
                flatten_2 = layers.Flatten()(CE_cnn_2)
                CE_dense_3 = layers.Dense(256, activation='relu')(flatten_2)
                #CE_dense_3 = layers.Dense(256, activation='relu')(CE_dense_3)
                CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE2')(CE_dense_3)
    

                reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])
                reconstructed_model.layers[1].trainable = False
                reconstructed_model.layers[2].trainable = False
                #reconstructed_model.layers[3].trainable = False


                lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=3e-2,
                decay_steps=10000,
                decay_rate=0.8)
                opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
                # Compile the models

                reconstructed_model.compile(optimizer=opt, 
                            loss = {'CE2' : 'categorical_crossentropy', 
                                      'mse' : 'mse'
                                     },              loss_weights=[1, 0],
                              metrics = {'CE2' : 'accuracy', 
                                         'mse': "mean_squared_error"
                                       }
                             )

                history1 = reconstructed_model.fit(x=x_train_2, y=(Y_train_2,x_train_2),
                                    batch_size=256,
                                    epochs=200,
                                    verbose=0,
                                    validation_data=(x_test_2, (Y_test_2,x_test_2)))

                results = reconstructed_model.evaluate(x = x_test_2, y = (Y_test_2,x_test_2))
                acc_list.append(results[3])

                savedata = {"lambda":lambda_val,
                            "first lr":0.05,
                            "first batchsize":256,
                            "first accuracy":first_acc,
                            "train snr":snr_train,
                            "test snr":snr_test,
                            "second loss":results[4],
                            "second accuracy":results[3]}

                datalist.append(savedata)

                print("final acc:", results[3])



current training snr:  0
current lambda:  0


2025-02-24 14:58:32.536250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78419 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2025-02-24 14:58:36.457879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2025-02-24 14:58:36.715418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-02-24 14:58:36.803341: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55f6826df5a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-24 14:58:36.803388: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2025-02-24 14:58:36.874972: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

current acc:  0.7142000198364258
training phase 2
current overlap:  0
test snr:  -5
157/157 [==============================] - 1s 3ms/step - loss: 1.0018 - CE2_loss: 1.0018 - mse_loss: 0.0741 - CE2_accuracy: 0.8104 - mse_mean_squared_error: 0.0741
final acc: 0.8104000091552734
test snr:  -2
157/157 [==============================] - 0s 2ms/step - loss: 1.0327 - CE2_loss: 1.0327 - mse_loss: 0.0741 - CE2_accuracy: 0.8068 - mse_mean_squared_error: 0.0741
final acc: 0.8068000078201294
test snr:  1
157/157 [==============================] - 0s 2ms/step - loss: 1.0104 - CE2_loss: 1.0104 - mse_loss: 0.0741 - CE2_accuracy: 0.8042 - mse_mean_squared_error: 0.0741
final acc: 0.8041999936103821
test snr:  4
157/157 [==============================] - 0s 2ms/step - loss: 1.0232 - CE2_loss: 1.0232 - mse_loss: 0.0741 - CE2_accuracy: 0.8060 - mse_mean_squared_error: 0.0741
final acc: 0.8059999942779541
test snr:  7
157/157 [==============================] - 0s 2ms/step - loss: 1.0557 - CE2_loss: 1.055

In [ ]:
from tensorflow.keras.models import load_model

embeddingDim = 80

batchsize = 256

for snr_train in [15,15,15]:
    print("current training snr: ",snr_train)
    for lambda_val in lambda_list:
        noise_sd = getnoisevariance(snr_train,1)
        print("current lambda: ",lambda_val )
        K.clear_session()
        tf.keras.utils.set_random_seed(42)

        input_layer  = layers.Input(shape=(32, 32, 3))
        encoder_1 = layers.Conv2D(96, (3, 3), activation='relu', padding='same')(input_layer)
        #maxpool_1 = layers.MaxPooling2D((2, 2))(encoder_1)
        encoder_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_1)
        maxpool_2 = layers.MaxPooling2D((2, 2))(encoder_2)
        #x = layers.Dropout(0.25)(maxpool_2)
        encoder_2_quantized = tf.keras.layers.Lambda(
            lambda x: tf.quantization.fake_quant_with_min_max_vars(x, min=0.0, max=6.0, num_bits=8)
        )(maxpool_2)
        
        embedding_network = tf.keras.Model(inputs=input_layer, outputs=encoder_2_quantized)

        normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2_quantized)
        noise_layer = GaussianNoiseLayer(stddev = noise_sd)(normalized_x)

        CE_cnn_1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
        flatten_1 = layers.Flatten()(CE_cnn_1)
        #bn = layers.BatchNormalization()(flatten_1)
        CE_dense_2 = layers.Dense(256, activation='relu')(flatten_1)
        #CE_dense_2 = layers.Dense(256, activation='relu')(CE_dense_2)
        #CE_dense_3 = layers.Dense(128, activation='relu')(CE_dense_2)
        #CE_dense_4 = layers.Dense(100, activation='relu')(CE_dense_3)

        CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_2)
        #CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

        #dense_2 = layers.Dense(64, activation='relu')(flatten_1)
        mse_1 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(noise_layer)
        upsample1 = layers.UpSampling2D((2,2))(mse_1)
        mse_2 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(upsample1)
        #upsample2 = layers.UpSampling2D((2,2))(mse_2)
        mse_output = layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same',name='mse')(mse_2)

        model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-2,
        decay_steps=10000,
        decay_rate=0.8)
        opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        model.compile(optimizer=opt,
                      loss = {'CE' : 'categorical_crossentropy', 
                              'mse' : 'mse'},
                      metrics = {'CE' : 'accuracy', 
                                 'mse': tf.keras.metrics.RootMeanSquaredError()},
                      loss_weights=[1, lambda_val])

        history = model.fit(x=x_train, y=(Y_train,x_train),
                            batch_size=256,
                            epochs=200,
                            verbose=0,
                            validation_data=(x_test, (Y_test,x_test)))
        
        first_acc = history.history['val_CE_accuracy'][-1]
        print("current acc: ",first_acc)
        print("training phase 2")
        
        embedding_network.save("embedding_network2.h5")


        #model.save("middle.h5")

        for i in range(label2_list.shape[0]):
            print("current overlap: ",(i))
            x_train_2,x_test_2,Y_train_2,Y_test_2 = get_data_phase1(label2_list[5-i])
            for snr_test in [15]: 
                print("test snr: ", snr_test)
                noise_sd_test = getnoisevariance(snr_test,1)
                
                #embedding_network_2 = load_model("embedding_network2.h5")
                embedding_network_2 = tf.keras.models.load_model("embedding_network2.h5", compile=False)
                for layer in embedding_network_2.layers:
                    layer.trainable = False
                    
                noise_layer_1 = GaussianNoiseLayer(stddev = noise_sd_test)(normalized_x)
                CE_cnn_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
                flatten_2 = layers.Flatten()(CE_cnn_2)
                CE_dense_3 = layers.Dense(256, activation='relu')(flatten_2)
                #CE_dense_3 = layers.Dense(256, activation='relu')(CE_dense_3)
                CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE2')(CE_dense_3)
    

                reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])
                reconstructed_model.layers[1].trainable = False
                reconstructed_model.layers[2].trainable = False
                #reconstructed_model.layers[3].trainable = False


                lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=3e-2,
                decay_steps=10000,
                decay_rate=0.8)
                opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
                # Compile the models

                reconstructed_model.compile(optimizer=opt, 
                            loss = {'CE2' : 'categorical_crossentropy', 
                                      'mse' : 'mse'
                                     },              loss_weights=[1, 0],
                              metrics = {'CE2' : 'accuracy', 
                                         'mse': "mean_squared_error"
                                       }
                             )

                history1 = reconstructed_model.fit(x=x_train_2, y=(Y_train_2,x_train_2),
                                    batch_size=256,
                                    epochs=200,
                                    verbose=0,
                                    validation_data=(x_test_2, (Y_test_2,x_test_2)))

                results = reconstructed_model.evaluate(x = x_test_2, y = (Y_test_2,x_test_2))
                acc_list.append(results[3])

                savedata = {"lambda":lambda_val,
                            "first lr":0.05,
                            "first batchsize":256,
                            "first accuracy":first_acc,
                            "train snr":snr_train,
                            "test snr":snr_test,
                            "second loss":results[4],
                            "second accuracy":results[3]}

                datalist.append(savedata)

                print("final acc:", results[3])



In [16]:

input_layer  = layers.Input(shape=(32, 32, 3))
encoder_1 = layers.Conv2D(96, (3, 3), activation='relu', padding='same')(input_layer)
#maxpool_1 = layers.MaxPooling2D((2, 2))(encoder_1)
encoder_2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_1)
maxpool_2 = layers.MaxPooling2D((2, 2))(encoder_2)
#x = layers.Dropout(0.25)(maxpool_2)
encoder_2_quantized = tf.keras.layers.Lambda(
    lambda x: tf.quantization.fake_quant_with_min_max_vars(x, min=0.0, max=6.0, num_bits=8)
)(maxpool_2)

embedding_network = tf.keras.Model(inputs=input_layer, outputs=encoder_2_quantized)

normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2_quantized)
noise_layer = GaussianNoiseLayer(stddev = 0.1)(normalized_x)

CE_cnn_1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(noise_layer)
flatten_1 = layers.Flatten()(CE_cnn_1)
#bn = layers.BatchNormalization()(flatten_1)
CE_dense_2 = layers.Dense(256, activation='relu')(flatten_1)
#CE_dense_2 = layers.Dense(256, activation='relu')(CE_dense_2)
#CE_dense_3 = layers.Dense(128, activation='relu')(CE_dense_2)
#CE_dense_4 = layers.Dense(100, activation='relu')(CE_dense_3)

CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_2)
#CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

#dense_2 = layers.Dense(64, activation='relu')(flatten_1)
mse_1 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(noise_layer)
upsample1 = layers.UpSampling2D((2,2))(mse_1)
mse_2 = layers.Conv2DTranspose(32, kernel_size=3, activation='relu', padding='same')(upsample1)
#upsample2 = layers.UpSampling2D((2,2))(mse_2)
mse_output = layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same',name='mse')(mse_2)

model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 32, 32, 96)   2688        ['input_3[0][0]']                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 32, 32, 32)   27680       ['conv2d_4[0][0]']               
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 16, 16, 32)  0           ['conv2d_5[0][0]']               
                                                                                            

In [ ]:
# Convert the dictionary to a DataFrame
import pandas as pd
df = pd.DataFrame(datalist)

# Define the file path and name
file_path = 'RLA_cifar_snr.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(file_path, index=False)

In [53]:
results = reconstructed_model.evaluate(x = x_test_2, y = (Y_test_2,x_test_2))

161/161 [==============================] - 1s 8ms/step - loss: 0.0553 - CE2_loss: 0.0553 - mse_loss: 0.2388 - CE2_accuracy: 0.9914 - mse_mean_squared_error: 0.2388


In [21]:
x = 1.9
tf.quantization.fake_quant_with_min_max_vars(x, min=0.0, max=6.0, num_bits=8)

<tf.Tensor: shape=(), dtype=float32, numpy=1.9058824>